<a href="https://colab.research.google.com/github/SumeyyeSuslu/regression/blob/main/elephas1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Jul 23 22:09:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install elephas

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
openjdk version "1.8.0_312"
OpenJDK Runtime Environ

In [ ]:
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam, SGD

from elephas.ml_model import ElephasEstimator
from elephas.ml.adapter import to_data_frame

from pyspark import SparkContext, SparkConf
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.ml.stat import Correlation
import pyspark.sql.functions as F


In [ ]:
# Create Spark context
#conf = SparkConf().setAppName('BostonHousing_Spark_MLP').setMaster('local[*]')
#sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext


In [ ]:

# Define basic parameters
batch_size = 32
epochs = 200

# Load data
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

404 train samples
102 test samples


In [ ]:


# Build RDD from numpy features and labels
df = to_data_frame(sc, x_train, y_train)
test_df = to_data_frame(sc, x_test, y_test)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
test_df.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.07092000544071...|  7.2|
|[4.83490206534042...| 18.8|
|[2.15568623389117...| 19.0|
|[0.00499396072700...| 27.0|
|[2.8043138445355E...| 22.2|
|[0.00109635305125...| 24.5|
|[1.19568627269472...| 31.2|
|[1.39254902023822...| 22.9|
|[3.64666688255965...| 20.5|
|[0.01399482414126...| 23.2|
|[8.97098041605204...| 18.6|
|[0.00531262764707...| 14.5|
|[0.00213537272065...| 17.8|
|[2.19686276977881...| 50.0|
|[0.04724784195423...| 20.8|
|[0.00133266672492...| 24.3|
|[3.46117652952671...| 24.2|
|[1.78196074557490...| 19.8|
|[0.02232058905065...| 19.1|
|[0.02039909735321...| 22.7|
+--------------------+-----+
only showing top 20 rows



In [ ]:
model = Sequential()
model.add(Dense(64, input_shape=(x_train.shape[1],)))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(1))

model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss='mean_absolute_error')

In [ ]:

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam(learning_rate=0.01)
opt_conf = optimizers.serialize(adam)
# Initialize Spark ML Estimator
estimator = ElephasEstimator()
estimator.set_keras_model_config(model.to_json())
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("mae")
estimator.set_metrics(['mse'])
estimator.set_epochs(epochs)
estimator.set_batch_size(batch_size)
estimator.set_validation_split(0.2)
estimator.set_categorical_labels(False)
estimator.set_verbosity(1)


# Fitting a model returns a Transformer
pipeline = Pipeline(stages=[estimator])
fitted_pipeline = pipeline.fit(df)



>>> Fit model
>>> Synchronous training complete.


In [ ]:
# Evaluate Spark model by evaluating the underlying model
prediction = fitted_pipeline.transform(test_df)
pnl = prediction.select("label", "prediction")
pnl.show(100)

prediction_and_label = pnl.rdd.map(lambda row: (row.label, row.prediction))


+-----+------------------+
|label|        prediction|
+-----+------------------+
|  7.2| 9.592015266418457|
| 18.8| 18.40625762939453|
| 19.0|23.382612228393555|
| 27.0|18.462295532226562|
| 22.2|23.327056884765625|
| 24.5|20.392885208129883|
| 31.2|28.624252319335938|
| 22.9| 25.51069450378418|
| 20.5| 19.16038703918457|
| 23.2|16.238985061645508|
| 18.6|10.045309066772461|
| 14.5|19.373628616333008|
| 17.8|16.967050552368164|
| 50.0|25.290498733520508|
| 20.8|13.900256156921387|
| 24.3|  24.1295166015625|
| 24.2|22.903528213500977|
| 19.8| 20.19230079650879|
| 19.1|15.845786094665527|
| 22.7| 16.67422866821289|
| 12.0|13.293987274169922|
| 10.2| 11.23094367980957|
| 20.0|21.200138092041016|
| 18.5|21.168771743774414|
| 20.9|30.929859161376953|
| 23.0|16.808128356933594|
| 27.5| 22.00514793395996|
| 30.1| 33.89569854736328|
|  9.5|10.948914527893066|
| 22.0|23.851518630981445|
| 21.2|19.665143966674805|
| 14.1|11.533576965332031|
| 33.1|29.070728302001953|
| 23.4| 22.36419105529785|
|

In [ ]:
metrics = RegressionMetrics(prediction_and_label)

print("RMSE: ",  metrics.rootMeanSquaredError)
print("r2: ", metrics.r2)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


RMSE:  7.659713819988884
r2:  -0.8848740204593828
